# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 5*. Работа с текстами: анализ тональности 

### Часть 1: подготовка данных

Импортируем библиотеку `pandas`: 

In [ ]:
import pandas as pd

Сейчас мы будем работать с готовым файлом `comments-as-rows.csv`, в котором сохранены все посты со стены [сообщества](https://vk.com/rzclimbing) скалодрома RockZona ВКонтакте:

In [ ]:
df = pd.read_csv("comments_as_rows.csv")
df.head()

Пояснения по названиям столбцов:
    
* `post_id`: id поста;
* `post_date`: дата публикации поста (POSIX формат);
* `post_text`: текст поста;
* `nlikes`: число лайков;
* `nreposts`: число репостов;
* `comm_id`: id комментария; 
* `comm_date`: дата публикации комментария (POSIX формат);
* `user_id`: id пользователя, оставившего комментарий;
* `comm_text`: текст комментария;
* `thread`: ответы на комментарий в виде словаря с разными характеристиками.

В полученном датафрейме одна строка соответствует одному комментарию, а не посту, поэтому id, тексты и прочие характеристики постов повторяются, а вот комментарии от строки к строке меняются. 

Оставим в датафрейме `df` только те строки, которые соответствуют постам с текстом. Можно удалить строки с пропусками, а мы, наоборот, выберем те строки, где значение `post_text` непустое, применив метод `.notnull()`:

In [ ]:
df = df[df["post_text"].notnull()]

Так как впереди нам предстоит работать исключительно с постами, давайте получим сокращённую версию датафрейма – избавимся от строк с повторяющимися значениями в столбце `post_id`, оставив только последнее из повторений (можно забрать и первое, отличия только в комментариях, а нас они пока не интересуют):

In [ ]:
df_post = df.drop_duplicates(subset = "post_id", keep = "last")

In [ ]:
df_post.head(3)

Отлично! Формат датафрейма подходит для дальнейшей задачи – анализа тональности постов.

### Часть 2: анализ тональности с билиотекой `dostoevsky`

Анализ тональности – определение эмоциональной окрашенности текста. С его помощью можно определить, позитивный ли перед нами текст, негативный или вообще нейтральный. 

Для анализа тональности мы будем использовать библиотеку `dostoevsky`. Эта библиотека разработана специально для русского языка, к тому же в неё входит модель, которая обучена на текстах из социальных сетей, что делает её особенно полезной для решения прикладных задач (всё-таки алгоритмы, которые учатся распознавать тональность текстов на твитах, постах или отзывах ближе к реальности, чем алгоритмы, обученные на стандартном корпусе литературных текстов).

Использовать библиотеку довольно просто, достаточно повторить действия, описанные в [документации](https://github.com/bureaucratic-labs/dostoevsky), однако с установкой библиотеки иногда возникают сложности. Проблема заключается в том, что эта библиотека зависит от библиотеки `fasttext`, которая используется для быстрой обработки текстов на разных языках, а она, в свою очередь, не работает на Windows без специальных дополнений, позволющих запускать код на C++ (собственно, именно из-за «ядра» на C++ всё быстро и работает).

Итак, порядок действий такой:

1. Попробовать запустить код `!pip install dostoevsky` для установки библиотеки в Jupyter. Если выводится ошибка, связанная с отсутствием/невозможностью установить `fasttext`, пробуем установить `fasttext` через `!pip install fasttext`. 

2. При установке `fasttext` Python выдаст сообщение об ошибке с актуальной ссылкой на Visual Studio C++ Build Tools. Если позволяет память компьютера, лучше установить все компоненты для простоты, но если хочется сократить список, можно выбрать только ключевые, см. перечень [здесь](https://medium.com/@oleg.tarasov/building-fasttext-python-wrapper-from-source-under-windows-68e693a68cbb) в *Step 2*.

3. После установки компонентов Visual Studio C++ Build Tools (ключевое – это компилятор кода на C++) снова пробуем установить `fasttext` через `!pip install fasttext` или сразу `dostoevsky` через  `!pip install dostoevsky`. Должно установиться! 

В крайнем случае можно установить `dostoevsky` в Google Colab, среда находится на сервере, работающим на Ubuntu, это Unix-система, не Windows.

Устанавливаем библиотеку:

In [ ]:
!pip install dostoevsky

После успешной установки скачиваем модель, которая будет предсказывать тональность текста:

In [ ]:
!python -m dostoevsky download fasttext-social-network-model

Импортируем классы `RegexTokenizer` и `FastTextSocialNetworkModel`, которые позволят создать токенизатор (инструмент для правильного разбиения текста на слова) и модель для предсказания тональности текста:

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

Класс – это довольно абстрактная вещь, это некоторый объект и его описание одновременно. Например, в библиотеке `pandas` ключевой объект – это датафрейм. Это класс, названный `DataFrame`, на котором разработчики библиотеки определили разные атрибуты и методы (и они же определили, как они работают). Так, с помощью функции `DataFrame()` мы создаём новый датафрейм, а затем можем применять к нему различные методы вроде `.describe()`, `.head()`, `.dropna()`. 

Здесь происходит то же самое, только создаются более непривычные объекты со своими методами и характеристиками:

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer = tokenizer)

Теперь, применяя метод `.predict()` к модели, которую мы определили ранее, мы сможем предсказать вероятности того, что текст поста относится к тому или иному типу. Всего в данной модели предусмотрено 5 типов:

* `positive`: положительно окрашенный;
* `negative`: отрицательно окрашенный;
* `neutral`: нейтральный;
* `speech`: приветствия, благодарности и подобные элементы речи;
* `skip`: неидентифицируемые случаи (реклама, шутки, цитаты, стихи);

Эти типы «унаследованы» от проекта [RuSentiment](https://github.com/text-machine-lab/rusentiment).

Мы выберем все 5 типов, потому что при меньшем числе типов будут отобраны самые вероятные, а они у разных постов будут разные.

In [ ]:
results = model.predict(df_post["post_text"], k = 5)

Так как модель уже готова и загружена, предсказание вероятностей происходит очень быстро. Посмотрим на первый результат – описание первого поста:

In [ ]:
print(results[0])

Результат представлен в виде словаря, где ключами являются названия типов, а значениями – вероятности, с которыми текст можно отнести к этому типу, согласно модели. Сохраним все эти вероятности в отдельный столбец датафрейма:

In [ ]:
df_post["results"] = results

А теперь по уже знакомой схеме извлечём из словаря вероятности для трёх самых понятных типов (положительный, отрицательный, нейтральный):

In [ ]:
df_post["positive"] = df_post["results"].apply(lambda x: x["positive"])
df_post["negative"] = df_post["results"].apply(lambda x: x["negative"])
df_post["neutral"] = df_post["results"].apply(lambda x: x["neutral"])

In [ ]:
df_post.head()

Теперь мы можем, например, сортировать посты по степени их «позитивности»:

In [ ]:
# топ 10 положительно окрашенных постов

df_post.sort_values("positive", ascending = False).head(10)

Или «негативности»:

In [ ]:
# топ 10 отрицательно окрашенных постов
# с определением негатива тут есть проблемы

df_post.sort_values("negative", ascending = False).head(10)